Unsupervised machine learning for Image generation using GAN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers

Importing the dataset from the local directory

In [ ]:
import warnings
from PIL import Image

# Suppress the warning about palette images with transparency
warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)
Image.warnings.simplefilter("ignore", Image.DecompressionBombWarning)

In [ ]:
#path to my data directory
main_folder = "/content/drive/MyDrive/Colab Notebooks/Generative ML/Emoji Dataset"
# Desired dimensions and color mode for the images
desired_dimension = (142, 107)
desired_color_mode = "RGBA"

# Initialize a list to store processed images
processed_images = []

# Traverse through subfolders and process images
for root, _, files in os.walk(main_folder):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(root, file)
            img = Image.open(image_path)
            img = img.resize(desired_dimension)
            img = img.convert(desired_color_mode)
            img_array = np.array(img)
            processed_images.append(img_array)

# Convert the list of processed images into a numpy array
images_array = np.array(processed_images)

# Print the shape of the resulting array
print("Shape of the array:", images_array.shape)

Shape of the array: (0,)


In [ ]:
type(images_array)

numpy.ndarray

In [ ]:
images_array[0]

IndexError: ignored

In [ ]:
plt.imshow(images_array[0])

In [ ]:
images_array= images_array.astype('float32')

In [ ]:
# Calculate the minimum and maximum values
min_value = np.min(images_array)
max_value = np.max(images_array)

In [ ]:
normalized_array = 2 * (images_array - min_value) / (max_value - min_value) - 1

print(normalized_array)

In [ ]:
plt.imshow(normalized_array[0].squeeze(), cmap = 'brg')

In [ ]:
buffer_size = 1664
batch_size = 64

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(normalized_array).shuffle(buffer_size).batch(batch_size)

In [ ]:
def generator_model():

    model = tf.keras.Sequential()
    model.add(layers.Dense(128, input_dim = 200))
    model.add(layers.ReLU())

    model.add(layers.Dense(256))
    model.add(layers.ReLU())

    model.add(layers.Dense(512))
    model.add(layers.ReLU())

    model.add(layers.Dense(107 * 142 * 3, activation = 'sigmoid'))
    model.add(layers.Reshape((107, 142, 3)))

    return model

In [ ]:
generator = generator_model()
generator.summary()

In [ ]:
noise = tf.random.normal([1, 200])
generated_image = generator(noise, training = False)

generated_image.shape

In [ ]:
plt.imshow(generated_image[0, :, :, 0], cmap = 'rainbow')

In [ ]:
def discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Input(shape = (107, 142, 3)))
    model.add(layers.Flatten())

    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Dense(1, activation = 'sigmoid'))

    return model

In [ ]:
discriminator = discriminator_model()
discriminator.summary()

In [ ]:
discriminator = discriminator_model()
output = discriminator(generated_image)
print (output)

In [ ]:
bce = tf.keras.losses.BinaryCrossentropy()

In [ ]:
def discriminator_loss(real_output, fake_output):

    real_loss = bce(tf.ones_like(real_output), real_output)

    fake_loss = bce(tf.zeros_like(fake_output), fake_output)

    total_loss = real_loss + fake_loss

    return total_loss

def generator_loss(fake_output):

    gen_loss = bce(tf.ones_like(fake_output), fake_output)

    return gen_loss

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001 )
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001 )

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(generator_optimizer = generator_optimizer,
                                 discriminator_optimizer = discriminator_optimizer,
                                 generator = generator,
                                 discriminator = discriminator)

In [ ]:
epochs = 50
noise_dim = 200
num_examples_to_generate = 16

seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training = True)

        real_output = discriminator(images, training = True)
        fake_output = discriminator(generated_images, training = True)

        disc_loss = discriminator_loss(real_output, fake_output)
        gen_loss = generator_loss(fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return (gen_loss, disc_loss, tf.reduce_mean(real_output), tf.reduce_mean(fake_output))

In [ ]:
def generate_and_plot_images(model, epoch, test_input):

    predictions = model(test_input, training = False)

    fig = plt.figure(figsize = (8, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        pred = (predictions[i, :, :, 0] + 1) * 127.5
        pred = np.array(pred)
        plt.imshow(pred.astype(np.uint8), cmap = 'rainbow')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [ ]:
def train(dataset, epochs):

    gen_loss_list = []
    disc_loss_list = []

    real_score_list =[]
    fake_score_list =[]
    for epoch in (range(epochs)):
        start = time.time()
        num_batches = len(dataset)

        print(f'Training started with  epoch {epoch + 1} with {num_batches} batches...')

        total_gen_loss = 0
        total_disc_loss = 0

        for batch in dataset:
            generator_loss, discriminator_loss, real_score, fake_score = train_step(batch)
            total_gen_loss += generator_loss
            total_disc_loss += discriminator_loss

        mean_gen_loss = total_gen_loss / num_batches
        mean_disc_loss = total_disc_loss / num_batches

        print('Losses after epoch %5d: generator %.3f, discriminator %.3f, real_score %.2f%%, fake_score %.2f%%'  %
              (epoch + 1, generator_loss, discriminator_loss, real_score * 100, fake_score * 100))

        generate_and_plot_images(generator, epoch + 1, seed)

        gen_loss_list.append(mean_gen_loss)
        disc_loss_list.append(mean_disc_loss)
        real_score_list.append(real_score)
        fake_score_list.append(fake_score)

        if (epoch + 1) % 10 == 0:
              checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))


    return  gen_loss_list, disc_loss_list, real_score_list, fake_score_list

In [ ]:
gen_loss_epochs, disc_loss_epochs, real_score_list, fake_score_list = train(train_dataset, epochs = epochs)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1, 2, figsize = (12, 8))

ax1.plot(gen_loss_epochs, label = 'Generator loss', alpha = 0.5)
ax1.plot(disc_loss_epochs, label = 'Discriminator loss', alpha = 0.5)
ax1.legend()

ax1.set_title('Training Losses')
ax2.plot(real_score_list, label = 'Real score', alpha = 0.5)
ax2.plot(fake_score_list, label = 'Fake score', alpha = 0.5)
ax2.set_title('Accuracy Scores')

ax2.legend()